In [1]:
from src.acoustic_analyser import frame
import numpy as np

test_frame = frame.from_file(
    member_file="test_member.json", constraint_file="test_constraint.csv", debug=False
)

free_end = test_frame.fixed_end(member_id=0)
free_end = test_frame.free_end(member_id=1)

In [2]:
test_frame.get_determinant(w=10)

(-0.22419369236865638-0.008377076802613428j)


(-0.22419369236865638-0.008377076802613428j)

In [3]:
# _ = test_frame.get_frequency_graph(lower_limit=1, upper_limit=100, step_size=2)

In [4]:
# test_frame.get_natural_frequency(lower_limit=10, upper_limit=12)

In [5]:
# test_frame.get_natural_frequency(lower_limit=50, upper_limit=55)

In [6]:
subs_dict = test_frame.get_params_solution(natural_freq=11.16)

In [57]:
from src.modules.bc import fixed_end

step_size = 0.1
constraint: fixed_end = test_frame.constraints[1]
member_curr = constraint.member
points = np.append(
    np.arange(step_size, member_curr.length, step_size), member_curr.length
)

In [58]:
v,u =member_curr.get_deformation(w=2*np.pi*11.16, length=points[0], id=constraint.id, subs_dict=subs_dict)

In [124]:
points=[0.1]

In [122]:
length = np.array(points)
w = 2*np.pi*11.16
C = (member_curr.youngs_modulus / member_curr.density) ** 0.5
K = (member_curr.inertia / member_curr.cross_section_area) ** 0.5

alpha = (w * K / C) ** 0.5
beta = w * K / C
L_bar = length / K

propagation_matrix_finder = lambda x : np.array(
    [
        [np.e** (-1j* alpha * x), 0, 0],
        [0, np.e** (-alpha * x), 0],
        [0, 0, np.e** (-1j* beta * x)],
    ]
)
propagation_matrix = np.stack([propagation_matrix_finder(x) for x in L_bar])
propagation_matrix_inv = np.linalg.inv(propagation_matrix)
a_plus_subs = np.matrix(member_curr.a_plus.subs(subs_dict),dtype=complex)
a_minus_subs = np.matrix(member_curr.a_minus.subs(subs_dict),dtype=complex)

v2 = (np.matmul(np.array([1,1,0]),propagation_matrix)*a_plus_subs + np.matmul(np.array([1,1,0]),propagation_matrix_inv)*a_minus_subs)
v2

matrix([[-0.01091331-0.50408612j]])

In [118]:
from sympy import Matrix
w = 2*np.pi*11.16
propagation_matrix_subs = member_curr.get_propagation_matrix(w=w,length=points[0])
propagation_matrix_inv_subs = propagation_matrix_subs.inv()
a_plus_subs = member_curr.a_plus.subs(subs_dict)
a_minus_subs = member_curr.a_minus.subs(subs_dict)

v = (Matrix([1,1,0]).T*propagation_matrix_subs*a_plus_subs + Matrix([1,1,0]).T*propagation_matrix_inv_subs*a_minus_subs).evalf()
u = (Matrix([0,0,1]).T*propagation_matrix_subs*a_plus_subs + Matrix([0,0,1]).T*propagation_matrix_inv_subs*a_minus_subs).evalf()


[[-4.59701721133854e-17 - 2.77555756156289e-17*I]]
